# Training the final model

This notebook is supplementary material to the project here, which aims to re-implement the Hubble.2d6 tool to predict the function of CYP2D6 star alleles.

Within this notebook, the final model in the sequence outlined in the paper is trained using transfer learning. Weights from the first step are loaded into the new model. The fully-connected layers of the first model are replaced with new, randomly initialised layers and trained on 31 star alleles and their respective suballeles.

Please keep in mind that this implementation is incomplete due to the lack of resources to compute robust annotation embeddings of all variants for all star alleles.

Additionally, in the paper, this model is supposed to instead have its weights - along with weights of one of the fully-connected layers - loaded in from the second model in the sequence. However, due to data availability, I was unable to reproduce the implementation of the second model and instead chose to transfer just the convolution layers' weights to the final model from the first model that was trained on simulated data.

## Getting ready

**Acknowledgements**: Pre-computed annotation embeddings used are form the original Hubble.2d6 repo: https://github.com/gregmcinnes/Hubble2D6/tree/master/data

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
np.random.seed(1337)

In [3]:
!git clone https://github.com/Locrian24/seng474-term-project.git
!cd seng474-term-project/ && git pull

fatal: destination path 'seng474-term-project' already exists and is not an empty directory.
Already up to date.


In [4]:
import sys
sys.path.insert(0, '/content/seng474-term-project')

## Preparing the data

The vcf file of the star alleles used were provided from the paper and can be found **here**. 

### Encoding method

The script `build_label_csv.py` was run on this vcf file to output the label csv that will be used for training.

Labels for the data correspond to the ordinal classes describing the function of each CYP2D6 star allele and their respective suballeles: "No function", "Decreased function", and "Normal function".

The model however is required to output two scores, representing the probability of a star allele being "No function" and "Normal function" respectively, and therefore a binary scoring system is used to encode the 3 function classes into these two scores. 

The scoring system is as follows:
- "No function" alleles are indicated with a 0 as the first score, with all other functions being denoted with a 1
- "Normal function" alleles are indicated with a 1 as the second score, with all other functions being denoted with a 0.

This yields the following encodings:
- "No function" = `[0, 0]`
- "Decreased function" = `[0, 1]`
- "Normal function" = `[1, 1]`

**Note**: Star alleles with uncurated function have an empty label (`[None, None]`). 

_Sequence encodings are done exactly the same as in step 1_

In [5]:
from encode_to_seq import Encode2Seq

# Global variables rn for testing

ANNOTATIONS = '/content/seng474-term-project/data/gvcf2seq.annotation_embeddings.csv'
EMBEDDINGS = '/content/seng474-term-project/data/embeddings.txt'
REF = '/content/seng474-term-project/data/ref.seq'

VCF = '/content/seng474-term-project/step3/star_samples.vcf'
LABELS = '/content/seng474-term-project/step3/labels.csv'

encoding = Encode2Seq(vcf=VCF, labels=LABELS, label_cols=[0, 1, 2], embedding_file=EMBEDDINGS, annotation_file=ANNOTATIONS, ref_seq=REF)

### Seperation of data sets

The paper indicates that they used 31 star alleles and their suballeles as training data, and 24 alleles to compose the test data. Additionally, 10% of the each functional class ("No function", "Decreased function", and "Normal function") were held out for validation during training.

All star alleles with "Uncurated function" were ignored, and held from training/testing for obvious reasons.

In [6]:
# Select only star alleles with curated function

mask = np.all(np.isnan(encoding.y) == False, axis=1)

sample_y = encoding.y[mask]
sample_X = encoding.X[mask.reshape(-1, 1).any(axis=1)]
sample_names = encoding.sample_names[mask]

In [7]:
# Get valid stars
all_stars = np.array([s.split('_')[1] for s in sample_names])
stars, idx = np.unique(all_stars, return_index=True)

# Choose which stars are training and which are test: (31, 24) split
# Should be stratified with labels
train_idx, test_idx = train_test_split(idx, stratify=sample_y[idx], test_size=24, random_state=0)

# Retrieve indices of training and test stars
sample_mask = np.isin(all_stars, all_stars[train_idx])

test_stars = np.array([s for s in sample_names[~sample_mask] if s.split('_')[-1] == 'vcf'])
test_mask = np.isin(sample_names, test_stars)

# Split the data into the two sets - INCLUDING SUBALLES ON BOTH
# _train_X, test_X = sample_X[sample_mask], sample_X[~sample_mask]
# _train_y, test_y = sample_y[sample_mask], sample_y[~sample_mask]

_train_X, test_X = sample_X[sample_mask], sample_X[test_mask]
_train_y, test_y = sample_y[sample_mask], sample_y[test_mask]

# Split training into train + validation (10% split -> validation)
train_X, val_X, train_y, val_y = train_test_split(_train_X, _train_y, stratify=_train_y, test_size=0.1, random_state=0)

In [8]:
# Uncurated star alleles

uncurated_samples = encoding.sample_names[~mask]
uncurated_stars = np.array([s for s in uncurated_samples if s.split('_')[-1] == 'vcf'])
uncurated_star_mask = np.isin(uncurated_samples, uncurated_stars)

# uncurated_X = encoding.X[uncurated_star_idx]
# uncurated_samples = uncurated_samples[uncurated_star_idx]

uncurated_samples = uncurated_samples[uncurated_star_mask]
uncurated_X = encoding.X[(~mask).reshape(-1, 1).any(axis=1)]
uncurated_X = uncurated_X[uncurated_star_mask]

### Prepare data for the model

In [9]:
_train_ds = tf.data.Dataset.from_tensor_slices((train_X, train_y))
train_ds = _train_ds.cache().shuffle(32).batch(32).prefetch(buffer_size=10)

_val_ds = tf.data.Dataset.from_tensor_slices((val_X, val_y))
val_ds = _val_ds.cache().shuffle(32).batch(32).prefetch(buffer_size=10)

_test_ds = tf.data.Dataset.from_tensors((test_X, test_y))
test_ds = _test_ds.cache().prefetch(buffer_size=10)

## Building the final model

As stated earlier, the model is loaded from the generated model from step 1, and also inherits its learned weights.

In [10]:
epochs = 20
fine_tune_epochs = 7

def build_and_fit(train_dataset, val_dataset):
  json_file = open('/content/seng474-term-project/step_1/model.json', 'r')
  loaded_model = json_file.read()
  model = tf.keras.models.model_from_json(loaded_model)
  model.load_weights('/content/seng474-term-project/step_1/weights.h5')
  
  # Remove fully connected layers
  model.pop()
  model.pop()
  model.pop()
  model.trainable = False

  # Build final model
  inputs = tf.keras.Input(shape=(14868, 13))
  model.training = False
  x = model(inputs, training=False)
  x = tf.keras.layers.Dense(32, activation=tf.keras.activations.relu, kernel_initializer=tf.keras.initializers.VarianceScaling(mode='fan_avg', distribution='uniform'), name = "dense_5")(x)
  x = tf.keras.layers.Dropout(rate=0.03, name="dropout_4")(x)
  x = tf.keras.layers.Dense(1, activation=tf.keras.activations.linear, kernel_initializer=tf.keras.initializers.VarianceScaling(mode='fan_avg', distribution='uniform'), name = "dense_6")(x)
  outputs = tf.keras.layers.Dense(2, activation=tf.keras.activations.sigmoid, kernel_initializer=tf.keras.initializers.VarianceScaling(mode='fan_avg', distribution='uniform'), name = "final_dense")(x)
  final_model = tf.keras.Model(inputs, outputs)

  # Initial training
  final_model.compile(
    optimizer=tf.keras.optimizers.Adam(0.01),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy()]
  )

  final_model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, verbose=False)

  # Fine tuning
  model.trainable = True
  final_model.compile(
      tf.keras.optimizers.Adam(1e-5),
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=[tf.keras.metrics.BinaryAccuracy()]
  )

  final_model.fit(train_dataset, epochs=fine_tune_epochs, validation_data=val_dataset, verbose=False)
  
  return final_model

## Training ensemble models

Hubble.2d6 uses an ensemble averaging method for it's final predictions. Here we are training 7 models to use in the ensemble, report the training accuracy of each and save these models for the final tool.

In [11]:
ensemble_size = 7

for i in range(ensemble_size):
  model = build_and_fit(train_ds, val_ds)
  model.evaluate(train_ds)

  # model.save(f'models/ensemble_{i}.model.h5')

4/4 [==============================] - 0s 81ms/step - loss: 0.1269 - binary_accuracy: 0.9440


## Model evaluation



First we compare the predictions generated by the model to the labels of the test set. Remember, these labels have been encoded to the scoring system and so may not adequately represent the output of the final model.

In [12]:
# Evaluation on encoded labels

model.evaluate(test_ds)

1/1 [==============================] - 0s 276ms/step - loss: 0.3825 - binary_accuracy: 0.8478


[0.38245582580566406, 0.8478260636329651]

Next we reverse the scoring system to each of the 3 functional classes using the cutoffs and conditions expressed within the paper.

In [13]:
preds = model.predict(test_X)

In [14]:
def get_functions(pred):
  cutpoint_1 = 0.4260022
  cutpoint_2 = 0.7360413

  cut1 = np.greater(pred[:, 0], [cutpoint_1])
  cut2 = np.greater(pred[:, 1], [cutpoint_2])

  functions = []
  for i in range(pred.shape[0]):
    if cut1[i] == True and cut2[i] == True:
      functions.append("Normal")
    elif cut1[i] == True and cut2[i] == False:
      functions.append("Decreased Function")
    else:
      functions.append("No function")

  return np.array(functions)

In [15]:
predictions = get_functions(preds)
true_labels = get_functions(test_y)

In [16]:
np.sum(predictions == true_labels) / len(true_labels)

0.7391304347826086

We return approximately around 75% accuracy on the 3 functional class labels. 

**Note**: This final prediction will be the output of the actual implementation.

## Predictions on "Uncurated function" star alleles

Here, we attempt predictions on the uncurated functional alleles that were present in the original dataset. Since the original paper also yields predictions on these alleles, I've taken it as a good guage at how similar the original Hubble.2d6 and my albeit incomplete model "think".

In [17]:
_uncurated_predictions = model.predict(uncurated_X)
uncurated_predictions = get_functions(_uncurated_predictions)

In [18]:
list(zip(uncurated_samples, uncurated_predictions))

[('CYP2D6_102_vcf', 'Normal'),
 ('CYP2D6_103_vcf', 'Normal'),
 ('CYP2D6_104_vcf', 'Decreased Function'),
 ('CYP2D6_105_vcf', 'Decreased Function'),
 ('CYP2D6_106_vcf', 'Decreased Function'),
 ('CYP2D6_107_vcf', 'Decreased Function'),
 ('CYP2D6_108_vcf', 'Normal'),
 ('CYP2D6_109_vcf', 'Decreased Function'),
 ('CYP2D6_110_vcf', 'Decreased Function'),
 ('CYP2D6_111_vcf', 'Decreased Function'),
 ('CYP2D6_112_vcf', 'Normal'),
 ('CYP2D6_113_vcf', 'Decreased Function'),
 ('CYP2D6_115_vcf', 'Decreased Function'),
 ('CYP2D6_116_vcf', 'Normal'),
 ('CYP2D6_117_vcf', 'No function'),
 ('CYP2D6_118_vcf', 'Normal'),
 ('CYP2D6_119_vcf', 'Decreased Function'),
 ('CYP2D6_120_vcf', 'No function'),
 ('CYP2D6_121_vcf', 'Normal'),
 ('CYP2D6_122_vcf', 'No function'),
 ('CYP2D6_123_vcf', 'Decreased Function'),
 ('CYP2D6_124_vcf', 'No function'),
 ('CYP2D6_125_vcf', 'No function'),
 ('CYP2D6_126_vcf', 'Decreased Function'),
 ('CYP2D6_127_vcf', 'Normal'),
 ('CYP2D6_128_vcf', 'No function'),
 ('CYP2D6_129_vcf', 